In [ ]:
import anthropic
import pandas as pd

In [ ]:
connections_df = pd.read_csv("./input/Connections.csv", skiprows=2)
# print(connections_df.head())

companies = connections_df.Company.unique()
# print(companies)
print(f"Successfully read-in connections. Total unique companies: {len(companies)}")

In [ ]:
api_key_file = open("./anthropic_api_key.txt", "r")
anthropic_api_key = api_key_file.read()

client = anthropic.Anthropic(api_key=anthropic_api_key)

In [ ]:
def query_claude(prompt, use_search=False):

    tools = []
    if use_search:
        tools = [
            {
                "name": "web_search",
                "type": "web_search_20250305",
                "max_uses": 1
            }
        ]
        betas = ["web-search-2025-03-05"]

    return client.beta.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=1024,
        temperature=1,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ],
        tools=tools
    )

In [ ]:
def loop_through_companies(companies, prompt_template, bool_col_name='IsGood'):
    
    result_df = pd.DataFrame(columns=['Company', bool_col_name, 'Message', 'IsErr'])
    total_input_tokens = 0
    total_output_tokens = 0
    counter = 0

    for company_name in companies:

        formatted_prompt = prompt_template.format(company_name=company_name)
        message = query_claude(formatted_prompt)

        final_text = message.content[-1].text
        bool_col = False
        is_err = False

        if final_text.endswith('TRUE'):
            bool_col = True
        elif final_text.endswith('FALSE'):
            bool_col = False
        else:
            is_err = True

        result_df.loc[len(result_df)] = [company_name, bool_col, final_text, is_err]

        total_input_tokens += message.usage.input_tokens
        total_output_tokens += message.usage.output_tokens
        counter += 1

        print(f"{counter:3d}/{len(companies)}: Processed company: {company_name}. Input tokens: {message.usage.input_tokens}; Output tokens: {message.usage.output_tokens}; Error: {is_err}")

    return result_df.set_index('Company')

In [ ]:
good_prompt = """
        I am looking for jobs. I am a data scientist looking for a company or non-profit working on a pro-social mission. 
        Some example cause areas: climate change, healthcare, preserving democracy, wealth inequality, education. 
        But I am interested in any others that are for the benefit of the greater good.
        Can you please let me know if this company fits that above description: {company_name}. 
        If it does meet this conditions, just reply "TRUE". If it does not, just reply "FALSE". Do not respond with the reasoning for this decision, 
        simply respond "TRUE" or "FALSE".
    """

result_df = loop_through_companies(companies, good_prompt, bool_col_name='IsGood')
result_df

In [ ]:
# result_df = pd.read_csv("./output/dave_connection_companies_first_pass.csv", index_col=0)

In [ ]:
good_df = result_df.loc[result_df['IsGood'] == True]
good_companies = good_df.index.tolist()
print(f"Good companies: {len(good_companies)}")

In [ ]:
local_prompt = """
        Can you please let me know if the company {company_name} is either fully remote or based in Colorado?
        I live in Colorado and can only work for a company that is based in Colorado or fully remote.
        If it is remote or in Colorado, just reply "TRUE". If it is neither, just reply "FALSE". 
        Do not respond with the reasoning for this decision, simply respond "TRUE" or "FALSE".
    """

loc_result_df = loop_through_companies(good_companies, local_prompt, bool_col_name='IsLocal')
loc_result_df

In [ ]:
loc_result_df.loc[loc_result_df['IsLocal'] == True]

In [ ]:
# HERE IS A NEW CELL

In [ ]:
loc_result_df.to_csv("./output/dave_connection_companies_second_pass.csv")
# result_df.to_csv("./output/dave_connection_companies_first_pass.csv", index=False)

In [ ]:
# class CompanyResult:
#     def __init__(self, is_good, message, is_err):
#         self.is_good = is_good
#         self.message = message
#         self.is_err = is_err

#     def __str__(self):
#         return f"CompanyResult(is_good={self.is_good}, message={self.message}, is_err={self.is_err})"

#     def __repr__(self):
#        return self.__str__()

#     def (self):
#         return {
#             "is_good": self.is_good,
#             "message": self.message,
#             "is_err": self.is_err
#         }

# "text": f"""
#     I am looking for jobs. I am a data scientist looking for a company or non-profit working on a pro-social mission. 
#     Some example cause areas: climate change, healthcare, preserving democracy, wealth inequality, education. 
#     But I am interested in any others that are for the benefit of the greater good. I can only work in Colorado or fully remotely. 
#     Can you please let me know if this company fits that above description: {company_name}. 
#     If it does meet these conditions, just reply "TRUE". If it does not, just reply "FALSE". Do not respond with the reasoning for this decision, 
#     simply respond "TRUE" or "FALSE".
# """